# Panderaで、read_csvで型を適用する

In [1]:
import pandera
import pandas as pd
import io

In [2]:
class Schema(pandera.DataFrameModel):
    h3: str
    lat: float
    lon: float
    group_id: int
    num: float

In [3]:
buf = io.BytesIO(b"""h3,lat,lon,group_id,num
882f5a32ddfffff,35.6812,139.7671,100,200
882f5a32defffff,35.6822,139.7681,101,202
""")
df = pd.read_csv(buf)
df.dtypes

h3           object
lat         float64
lon         float64
group_id      int64
num           int64
dtype: object

In [4]:
# 自動ではINTと見なされる
df.num.dtype

dtype('int64')

In [5]:
# そのままでは、validateするとエラーになる
try:
    Schema.validate(df)
except Exception as e:
    print(e)

expected series 'num' to have type float64, got int64


In [6]:
# dtypesはto_scema()すると取れる
Schema.to_schema().dtypes

{'h3': DataType(str),
 'lat': DataType(float64),
 'lon': DataType(float64),
 'group_id': DataType(int64),
 'num': DataType(float64)}

In [9]:
buf = io.BytesIO(b"""h3,lat,lon,group_id,num
882f5a32ddfffff,35.6812,139.7671,100,200
882f5a32defffff,35.6822,139.7681,101,202
""")
# 一度変換が必要
_dtype = {col: str(dtype) for col, dtype in Schema.to_schema().dtypes.items()}
df = pd.read_csv(buf, dtype=_dtype)
df.dtypes

h3           object
lat         float64
lon         float64
group_id      int64
num         float64
dtype: object

In [10]:
# 型エラーになるケース
buf = io.BytesIO(b"""h3,lat,lon,group_id,num
882f5a32ddfffff,35.6812,139.7671,100,200bc
882f5a32defffff,35.6822,139.7681,101,202db
""")
_dtype = {col: str(dtype) for col, dtype in Schema.to_schema().dtypes.items()}
try:
    df = pd.read_csv(buf, dtype=_dtype)
except Exception as e:
    print(e)


could not convert string to float: '200bc'


メモ: pandera_toolsというモジュールがあったが、あまりStarがなく、メンテされているか自信が無かったので利用しない方針